In [ ]:
import os
import glob
from skimage import io
import matplotlib.pyplot as plt

from skimage.measure import regionprops
from skimage.morphology import label
import numpy as np

In [ ]:
labels_path = r'D:\Institute of Genetics and Cancer\Neurosphere assay\Assess_Segmentation_Data\Ground_Truth'
images_path = r'D:\Institute of Genetics and Cancer\Neurosphere assay\Assess_Segmentation_Data\Images'

all_images =[]
for file in glob.glob(images_path + "\*.tif"):
    all_images.append(os.path.basename(file))

In [ ]:
# Import test image
#image_name = 'D - 03.tif'

#img_filename = os.path.join(images_path, image_name)
#lbl_filename = os.path.join(labels_path, image_name)
#image = io.imread(img_filename)
#labels = io.imread(lbl_filename)

# Get labels and plot
#labels = label(labels)

#fig, ax = plt.subplots(1, 3, figsize=(16, 8))
#ax[0].imshow(image, cmap='gray')
#ax[1].imshow(labels, cmap='nipy_spectral', interpolation='none')
#ax[2].imshow(image, cmap='gray')
#ax[2].imshow(labels, cmap='nipy_spectral', alpha=.5, interpolation='none')
#fig.tight_layout
#plt.show()

In [ ]:
def calculate_regionprops(label_image):
    import math

    # Define additiinal properties
    def circularity(p):
        return 4 * math.pi * p['area'] / (p['perimeter_crofton'] ** 2) # 'perimeter' calculates circularities up to 1.5... crofton seems to be more accurate
    def aspect_ratio(p):
        return p['axis_major_length'] / p['axis_minor_length']

    # Get properties as numpy arrays
    props = regionprops(label_image)
    area_np = np.asarray([p['area'] for p in props])
    solidity_np = np.asarray([p['solidity'] for p in props])
    circularity_np = np.asarray([circularity(p) for p in props])
    aspect_ratio_np = np.asarray([aspect_ratio(p) for p in props])
    return area_np, solidity_np, circularity_np, aspect_ratio_np

## Plot histograms for the different shape descriptors

In [ ]:
area_array = []
solidity_array = []
circularity_array = []
aspect_ratio_array = []
for image_name in all_images:
    # Load original image
    #img_filename = os.path.join(images_path, image_name)
    #image = io.imread(img_filename)
        
    # Load label image
    lbl_filename = os.path.join(labels_path, image_name)
    labels = io.imread(lbl_filename)
    
    # Transform to label matrix
    labels = label(labels)

    area, sol, circ, as_re = calculate_regionprops(labels)
    area_array = np.concatenate((area_array, area))
    solidity_array = np.concatenate((solidity_array, sol))
    circularity_array = np.concatenate((circularity_array, circ))
    aspect_ratio_array = np.concatenate((aspect_ratio_array, as_re))

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Object distribution for different shape descriptors', fontsize=16)
ax[0,0].set_title('Area')
ax[0,0].set_ylabel('Number of objects')
ax[0,0].hist(area_array, bins=40)
ax[0,1].set_title('Solidity')
ax[0,1].set_ylabel('Number of objects')
ax[0,1].hist(solidity_array, bins=40)
ax[1,0].set_title('Circularity')
ax[1,0].set_ylabel('Number of objects')
ax[1,0].hist(circularity_array, bins=40)
ax[1,1].set_title('Aspect Ratio')
ax[1,1].set_ylabel('Number of objects')
ax[1,1].hist(aspect_ratio_array, bins=40)
plt.show()

## Create color-coded images for the different shape descriptors

In [ ]:
def feature_mask(label_image, feature_list):
    class_list = []

    props = regionprops(label_image)
    for i in range(len(props)):
        label_id = props[i].label # gets the label id of the current region
        
        if(feature_list[i]<500):
            class_mask = np.where(label_image == label_id, 1, 0)
        elif(feature_list[i]<2000):
            class_mask = np.where(label_image == label_id, 2, 0)
        elif(feature_list[i]<4000):
            class_mask = np.where(label_image == label_id, 3, 0)
        else:
            class_mask = np.where(label_image == label_id, 4, 0)
        
        class_list.append(class_mask)

    # reconstructs the label image, now containing the eroded labels
    class_stack = np.stack(class_list) # creates stack from list of images (numpy arrays)
    feature_labels = np.max(class_stack, axis = 0) # calculates the maximum projection to get back a 2D, labelled image
    return feature_labels

In [ ]:
def color_code_image(image, label_image):
    from skimage.color import label2rgb

    # define RGB colors from Tableau 10 (default in matplotlib)
    t10_blue = [31/255, 119/255, 180/255]
    t10_orange = [255/255, 127/255, 14/255]
    t10_green = [44/255, 160/255, 44/255]
    t10_red = [214/255, 39/255, 40/255]

    # list of colors
    colors=[t10_blue, t10_orange, t10_green, t10_red]

    # create RGB image with color-labels over the original, grayscale image
    rgb_labels = label2rgb(
        label=label_image,
        image=image,
        colors=colors,
        alpha=0.7,
        bg_label=0,
        bg_color=None
        )

    # plot original and color-coded images
    fig, ax = plt.subplots(1,2, figsize=(16,12))
    ax[0].imshow(image, cmap='gray')
    ax[1].imshow(rgb_labels)
    plt.show()

In [ ]:
for image_name in all_images:
    
    # Load original image
    img_filename = os.path.join(images_path, image_name)
    image = io.imread(img_filename)
    
    # Load label image
    lbl_filename = os.path.join(labels_path, image_name)
    labels = io.imread(lbl_filename)

    # Transform to label matrix
    labels = label(labels)

    # Calculate region properties
    area, sol, circ, as_re = calculate_regionprops(labels)

    # Get feature labels
    color_code_area = feature_mask(labels, area)

    # plot color-coded images
    color_code_image(image, color_code_area)